In [1]:
import pandas as pd
import datetime as dt
import csv

In [2]:
# trial with 10
data = pd.read_csv('../data/raw/journey.csv')
data.shape

(354634, 11)

In [3]:
# remove rows with start and end stations overlapping.
data = data[data['Start station number'] != data['End station number']]
data.shape

(346130, 11)

In [4]:
# trial with 10
data10 = data.head(1000).copy()
# remaking dates
data10_dates = pd.to_datetime(data10['Start date'])
data10['Start date'] = data10_dates.dt.strftime('%Y%m%d')
data10['Start time'] = data10_dates.dt.strftime('%H%M')
data10 = data10.drop(['Number', 'Bike number', 'Total duration', 'End date', 'Start station', 'End station'], axis=1)
data10['Total duration (ms)'] = pd.to_numeric(data10['Total duration (ms)']) // (1000 * 60)
data10 = data10.rename(columns={'Total duration (ms)': 'Total duration (m)'})
data10.head()

,Start date,Start station number,End station number,Bike model,Total duration (m),Start time
0,20250131,1142,988,CLASSIC,36,2359
1,20250131,1142,988,CLASSIC,36,2359
2,20250131,300061,300024,CLASSIC,20,2359
3,20250131,300058,300076,CLASSIC,4,2359
4,20250201,300038,300037,CLASSIC,12,0000


In [5]:
stations_data = pd.read_xml('../data/raw/livecyclehireupdates.xml')
stations_data['station_coordinates'] = stations_data['lat'].astype(str) + ',' + stations_data['long'].astype(str)
stations_data = stations_data.drop(['id', 'name', 'lat', 'long', 'installed', 'locked', 'installDate', 'removalDate', 'temporary', 'nbBikes', 'nbStandardBikes', 'nbEBikes', 'nbEmptyDocks', 'nbDocks'], axis=1)
stations_data.head()

,terminalName,station_coordinates
0,1023,"51.52916347,-0.109970527"
1,1018,"51.49960695,-0.197574246"
2,1012,"51.52128377,-0.084605692"
3,1013,"51.53005939,-0.120973687"
4,3420,"51.49313,-0.156876"


In [6]:
merged_data10 = data10.merge(
    stations_data,
    left_on='Start station number',
    right_on='terminalName'
)
merged_data10 = merged_data10.rename(columns={'station_coordinates': 'start_coordinates'})

merged_data10 = merged_data10.merge(
    stations_data,
    left_on='End station number',
    right_on='terminalName'
)
merged_data10 = merged_data10.rename(columns={'station_coordinates': 'end_coordinates'})

merged_data10 = merged_data10.drop(['terminalName_x', 'terminalName_y'], axis=1)
merged_data10

,Start date,Start station number,End station number,Bike model,Total duration (m),Start time,start_coordinates,end_coordinates
0,20250131,1142,988,CLASSIC,36,2359,"51.5034938,-0.07962099","51.51772703,-0.127854211"
1,20250131,1142,988,CLASSIC,36,2359,"51.5034938,-0.07962099","51.51772703,-0.127854211"
2,20250131,300061,300024,CLASSIC,20,2359,"51.495598,-0.078893","51.53658514,-0.075885686"
3,20250131,300058,300076,CLASSIC,4,2359,"51.48512191,-0.174971902","51.4795738,-0.17903854"
4,20250201,300038,300037,CLASSIC,12,0000,"51.48724429,-0.205279052","51.49422354,-0.236769936"
...,...,...,...,...,...,...,...,...
982,20250131,1006,3439,CLASSIC,8,2147,"51.51863,-0.1326763000000141","51.5262503,-0.123509611"
983,20250131,300031,200017,CLASSIC,41,2147,"51.53638435,-0.102757578","51.511654,-0.179668"
984,20250131,1225,300097,CLASSIC,21,2145,"51.517703,-0.154106","51.491026,-0.209121"
985,20250131,972,300235,CLASSIC,12,2145,"51.52554222,-0.138231303","51.541596,-0.12544145"


In [7]:
merged_data10.to_csv('../data/processed/processed1000.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)